In [166]:
import tensorflow as tf
import pandas as pd 
import numpy as np 

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

import seaborn as sns

In [216]:
df = pd.read_csv("../data/raw/clean_data.csv")
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Unemployment,Type,Size,Super_Bowl,Labor_Day,Thanksgiving,Christmas,week,month,year
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.0,0.0,0.0,...,8.106,A,151315,False,False,False,False,5,2,2010
1,1,2,2010-02-05,50605.27,False,42.31,2.572,0.0,0.0,0.0,...,8.106,A,151315,False,False,False,False,5,2,2010
2,1,3,2010-02-05,13740.12,False,42.31,2.572,0.0,0.0,0.0,...,8.106,A,151315,False,False,False,False,5,2,2010
3,1,4,2010-02-05,39954.04,False,42.31,2.572,0.0,0.0,0.0,...,8.106,A,151315,False,False,False,False,5,2,2010
4,1,5,2010-02-05,32229.38,False,42.31,2.572,0.0,0.0,0.0,...,8.106,A,151315,False,False,False,False,5,2,2010


In [217]:
df['Date'] = pd.to_datetime(df["Date"])
df.index = df['Date']
df = df.drop(['Date'], axis=1)
df['Weekly_Sales'] = np.log(df['Weekly_Sales'])
train = df.loc[:'2012-10-01'].reset_index()
test = df.loc['2012-10-01':].reset_index()
train.shape, test.shape

/var/folders/1q/26x3nv656dg2gv4xzbytws5m0000gn/T/ipykernel_35779/691307098.py:6: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  test = df.loc['2012-10-01':].reset_index()


((408369, 23), (11843, 23))

In [218]:
train = train.set_index("Date")
train['Weekly_sales_lag'] = train['Weekly_Sales'].shift(4)
train = train.reset_index()
train = train.dropna()

test = test.set_index("Date")
test['Weekly_sales_lag'] = test['Weekly_Sales'].shift(4)
test = test.reset_index()
test = test.dropna()


In [219]:
X_train = train.drop(['Weekly_Sales'], axis=1)
y_train = train['Weekly_Sales']

X_test = test.drop(['Weekly_Sales'], axis=1)
y_test = test['Weekly_Sales']

In [220]:
# X_train['Date'] = pd.to_datetime(X_train['Date'])

In [221]:
X_train.columns

Index(['Date', 'Store', 'Dept', 'IsHoliday', 'Temperature', 'Fuel_Price',
       'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI',
       'Unemployment', 'Type', 'Size', 'Super_Bowl', 'Labor_Day',
       'Thanksgiving', 'Christmas', 'week', 'month', 'year',
       'Weekly_sales_lag'],
      dtype='object')

In [222]:
CATEGORICAL_FEATURE_KEYS = [
    # 'Dept',
    'IsHoliday',
    "Super_Bowl",
    "Type",
    "Size",
    "Labor_Day",
    "Thanksgiving",
    "Christmas",
    "year",
    "week"
]

NUMERIC_FEATURE_KEYS = [
    "Temperature",
    "Fuel_Price",
    "MarkDown1",
    "MarkDown2",
    "MarkDown3",
    "MarkDown4",
    "MarkDown5",
    "CPI",
    "Weekly_sales_lag"
]

# ORDINAL_FEATURE_KEYS = [
#     "year",
#     "week"
# ]

In [223]:
ct = ColumnTransformer(
    [("Numeric", StandardScaler(), NUMERIC_FEATURE_KEYS),
     ("Categorical", OneHotEncoder(), CATEGORICAL_FEATURE_KEYS)
    ])




In [224]:
pipeline = Pipeline(steps=[('preprocessor', ct)])
X_train_trans = ct.fit_transform(X_train)
X_test_trans = ct.transform(X_test)
trans_col = pipeline.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(CATEGORICAL_FEATURE_KEYS)

In [225]:
X_train_trans = pd.DataFrame(X_train_trans.toarray(), columns=NUMERIC_FEATURE_KEYS + trans_col.tolist())
X_test_trans = pd.DataFrame(X_test_trans.toarray(),columns=NUMERIC_FEATURE_KEYS + trans_col.tolist())
# X_train_trans['week'] = X_train['week']
# X_test_trans['week'] = X_test['week']
# X_train_trans = X_train_trans.dropna()
# X_test_trans = X_test_trans.dropna()



In [226]:
X_train_trans

,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Weekly_sales_lag,IsHoliday_False,...,week_43,week_44,week_45,week_46,week_47,week_48,week_49,week_50,week_51,week_52
0,-0.953080,-1.702558,-0.419353,-0.175588,-0.085799,-0.274567,-0.380216,1.023768,0.781635,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.953080,-1.702558,-0.419353,-0.175588,-0.085799,-0.274567,-0.380216,1.023768,1.127018,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.953080,-1.702558,-0.419353,-0.175588,-0.085799,-0.274567,-0.380216,1.023768,0.491200,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.953080,-1.702558,-0.419353,-0.175588,-0.085799,-0.274567,-0.380216,1.023768,1.011765,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.953080,-1.702558,-0.419353,-0.175588,-0.085799,-0.274567,-0.380216,1.023768,0.906984,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408360,0.258408,1.429542,0.325591,-0.171579,-0.085531,0.131404,0.398139,0.535703,0.273228,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
408361,0.258408,1.429542,0.325591,-0.171579,-0.085531,0.131404,0.398139,0.535703,0.733156,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
408362,0.258408,1.429542,0.325591,-0.171579,-0.085531,0.131404,0.398139,0.535703,0.534664,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
408363,0.258408,1.429542,0.325591,-0.171579,-0.085531,0.131404,0.398139,0.535703,1.094820,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [227]:
X_train_trans.shape, X_train_trans.columns

((408365, 117),
 Index(['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
        'MarkDown4', 'MarkDown5', 'CPI', 'Weekly_sales_lag', 'IsHoliday_False',
        ...
        'week_43', 'week_44', 'week_45', 'week_46', 'week_47', 'week_48',
        'week_49', 'week_50', 'week_51', 'week_52'],
       dtype='object', length=117))

In [228]:
trans_col

array(['IsHoliday_False', 'IsHoliday_True', 'Super_Bowl_False',
       'Super_Bowl_True', 'Type_A', 'Type_B', 'Type_C', 'Size_34875',
       'Size_37392', 'Size_39690', 'Size_39910', 'Size_41062',
       'Size_42988', 'Size_57197', 'Size_70713', 'Size_93188',
       'Size_93638', 'Size_103681', 'Size_112238', 'Size_114533',
       'Size_118221', 'Size_119557', 'Size_120653', 'Size_123737',
       'Size_125833', 'Size_126512', 'Size_128107', 'Size_140167',
       'Size_151315', 'Size_152513', 'Size_155078', 'Size_155083',
       'Size_158114', 'Size_184109', 'Size_196321', 'Size_200898',
       'Size_202307', 'Size_202505', 'Size_203007', 'Size_203742',
       'Size_203750', 'Size_203819', 'Size_204184', 'Size_205863',
       'Size_206302', 'Size_207499', 'Size_219622', 'Labor_Day_False',
       'Labor_Day_True', 'Thanksgiving_False', 'Thanksgiving_True',
       'Christmas_False', 'Christmas_True', 'year_2010', 'year_2011',
       'year_2012', 'week_1', 'week_2', 'week_3', 'week_4', 'we

In [229]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np 

In [230]:
lr = LinearRegression()
lr.fit(X_train_trans, y_train)

LinearRegression()

In [231]:
pred_lr = lr.predict(X_train_trans)

In [232]:
print("Train")
np.sqrt(mean_squared_error(y_train, pred_lr))

Train


1.9049469796673701

In [233]:
print("test")
test_pred_lr = lr.predict(X_test_trans)
np.sqrt(mean_squared_error(y_test, test_pred_lr))

test


1.8262898697153267

In [234]:
y_test.min(), y_test.max(), y_test.std()

(-4.605170185988091, 12.169724223396777, 1.9976587043296536)

In [197]:
rt = RandomForestRegressor(max_depth= 50)
rt.fit(X_train_trans, y_train)

RandomForestRegressor(max_depth=50)

In [235]:
pred_rt = rt.predict(X_train_trans)

In [236]:
test_pred_rt = rt.predict(X_test_trans)

In [237]:
np.sqrt(mean_squared_error(y_train, pred_rt))

0.7618383334851071

In [238]:
y_train.min(), y_train.max(), y_train.std()

(-4.605170185988091, 13.448928644517972, 2.050487503404155)

In [239]:
np.sqrt(mean_squared_error(y_test, test_pred_rt))

1.7663920528199546

In [240]:
y_test.min(), y_test.max(), y_test.std()

(-4.605170185988091, 12.169724223396777, 1.9976587043296536)

In [241]:
import tensorflow as tf 

In [305]:
class RNNModel:
    def __init__(self, rnn_units=100, return_sequences=False):
        # Your code here
        self.model = tf.keras.Sequential(
            [
                # Batch_size, rnn_units. When return_sequence = true, shape = N, input_shape, rnn_units. (useful for stacking RNN)
                tf.keras.layers.LSTM(rnn_units, return_sequences=return_sequences),
                # tf.keras.layers.SimpleRNN(128),
                tf.keras.layers.Dense(10),
                tf.keras.layers.Dense(5),
                tf.keras.layers.Dense(1),
                tf.keras.layers.Dense(1, activation='linear'),
            ]
        )
        self.model.compile(
            optimizer=tf.keras.optimizers.RMSprop(),
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[
                tf.keras.metrics.mean_squared_error,
                tf.keras.metrics.mean_absolute_error,
            ],
        )

    def fit(self, train_data, epochs, val_data=None, callbacks=None):
        # Your code here
        history = self.model.fit(
            train_data, epochs=epochs, validation_data=val_data, callbacks=callbacks
        )
        return history

    def evaluate(self, eval_data, verbose=0):
        # Your code here
        result = self.model.evaluate(eval_data, verbose=verbose, return_dict=True)
        return result['mean_squared_error'], result['mean_absolute_error']

    def predict(self, pred_data):
        # Your code here
        return self.model.predict(pred_data)

In [306]:
model = RNNModel()

In [307]:
# model = tf.keras.Sequential(
#             [
#                 # Batch_size, rnn_units. When return_sequence = true, shape = N, input_shape, rnn_units. (useful for stacking RNN)
#                 tf.keras.layers.LSTM(32, return_sequences=False),
#                 tf.keras.layers.Dense(1),
#             ]
#         )
# model.compile(
#     optimizer=tf.keras.optimizers.RMSprop(),
#     loss=tf.keras.losses.MeanSquaredError(),
#     metrics=[
#         tf.keras.metrics.mean_squared_error,
#         tf.keras.metrics.mean_absolute_error,
#     ],
# )

In [308]:
class WindowGenerator:
    def __init__(
        self, lookback, lookahead, batch_size, train_df, test_df, label_column
    ):
        # Your code here
        # Store data
        self.train_df = train_df
        self.test_df = test_df
        # Get the column indices
        self.label_column = label_column
        self.label_column_indices = {
            name: i for i, name in enumerate(label_column)
        }  # Encoding column name into index
        self.column_indices = {name: i for i, name in enumerate(self.train_df.columns)}

        # Window parameters
        self.lookback = lookback
        self.lookahead = lookahead
        self.total_window_size = self.lookback + self.lookahead
        self.input_slice = slice(0, self.lookback)
        self.lookback_idx = np.arange(self.total_window_size)[self.input_slice]
        self.label_start = self.total_window_size - self.lookahead
        self.label_slice = slice(self.label_start, None)
        self.label_idx = np.arange(self.total_window_size)[self.label_slice]

        self.batch_size = batch_size

    def __repr__(self):
        return "\n".join(
            [
                f"Total window size: {self.total_window_size}",
                f"Lookback indices: {self.lookback_idx}",
                f"Label index: {self.label_idx}",
                f"Label name: {self.label_column}",
            ]
        )

    @property
    def train(self):
        return self.make_dataset(self.train_df)


    @property
    def test(self):
        return self.make_dataset(self.test_df, shuffle=False)

    def make_dataset(self, data, shuffle=True):
        # Your code here
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=shuffle,
            batch_size=32,
        )

        ds = ds.map(self.split_window)

        return ds

    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.label_slice, :]
        labels = tf.stack(
            [labels[:, :, self.column_indices[name]] for name in self.label_column],
            axis=-1,
        )
        inputs.set_shape([None, self.lookback, None])
        labels.set_shape([None, self.lookahead, None])

        return inputs, labels


In [309]:
train_1 = X_train_trans
train_1['Date'] = train.Date
train_1['Weekly_Sales'] = y_train
train_1 = train_1.set_index(['Date'])
train_1 = train_1.dropna()

In [310]:
test_1 = X_test_trans
test_1['Date'] = test.Date
test_1['Weekly_Sales'] = y_test
test_1 = test_1.set_index(['Date'])
test_1 = test_1.dropna()

In [311]:
train_1

,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Weekly_sales_lag,IsHoliday_False,...,week_44,week_45,week_46,week_47,week_48,week_49,week_50,week_51,week_52,Weekly_Sales
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-05,-0.953080,-1.702558,-0.419353,-0.175588,-0.085799,-0.274567,-0.380216,1.023768,0.906984,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.380634
2010-02-05,-0.953080,-1.702558,-0.419353,-0.175588,-0.085799,-0.274567,-0.380216,1.023768,0.066282,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.656786
2010-02-05,-0.953080,-1.702558,-0.419353,-0.175588,-0.085799,-0.274567,-0.380216,1.023768,0.700028,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.956274
2010-02-05,-0.953080,-1.702558,-0.419353,-0.175588,-0.085799,-0.274567,-0.380216,1.023768,1.013896,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.599855
2010-02-05,-0.953080,-1.702558,-0.419353,-0.175588,-0.085799,-0.274567,-0.380216,1.023768,0.593042,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.736901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-09-28,0.258408,1.429542,0.325591,-0.171579,-0.085531,0.131404,0.398139,0.535703,0.273228,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.081125
2012-09-28,0.258408,1.429542,0.325591,-0.171579,-0.085531,0.131404,0.398139,0.535703,0.733156,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.024203
2012-09-28,0.258408,1.429542,0.325591,-0.171579,-0.085531,0.131404,0.398139,0.535703,0.534664,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.617197


In [312]:
w1 = WindowGenerator(lookback=6, lookahead=1, batch_size=100, train_df=train_1, test_df=test_1, label_column=['Weekly_Sales'])

In [313]:
history = model.fit(w1.train, epochs=10)

Epoch 1/10


2022-12-15 16:33:09.632787: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 16:33:09.931868: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 16:33:10.339000: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


12762/12762 [==============================] - 254s 20ms/step - loss: 2.2298 - mean_squared_error: 2.2298 - mean_absolute_error: 1.0102
Epoch 2/10
12762/12762 [==============================] - 249s 20ms/step - loss: 1.4121 - mean_squared_error: 1.4121 - mean_absolute_error: 0.7335
Epoch 3/10
12762/12762 [==============================] - 252s 20ms/step - loss: 1.2308 - mean_squared_error: 1.2308 - mean_absolute_error: 0.6596
Epoch 4/10
12762/12762 [==============================] - 252s 20ms/step - loss: 1.1401 - mean_squared_error: 1.1401 - mean_absolute_error: 0.6209
Epoch 5/10
12762/12762 [==============================] - 252s 20ms/step - loss: 1.0779 - mean_squared_error: 1.0779 - mean_absolute_error: 0.5957
Epoch 6/10
12762/12762 [==============================] - 282s 22ms/step - loss: 1.0319 - mean_squared_error: 1.0319 - mean_absolute_error: 0.5771
Epoch 7/10
12762/12762 [==============================] - 255s 20ms/step - loss: 0.9983 - mean_squared_error: 0.9983 - mean_absol

In [314]:
pred = model.predict(w1.train)

    1/12762 [..............................] - ETA: 1:41:54

2022-12-15 17:16:31.169802: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 17:16:31.270068: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


12762/12762 [==============================] - 75s 6ms/step


In [318]:
np.sqrt(mean_squared_error(train_1['Weekly_Sales'][6:], pred.flatten()))

2.765172022365282

In [323]:
np.sqrt(model.evaluate(w1.train)[1])

0.7353736499240329

In [324]:
np.sqrt(model.evaluate(w1.test)[1])

0.7669559121447977

In [321]:
pred = model.predict(w1.test)

370/370 [==============================] - 3s 8ms/step


In [322]:
np.sqrt(mean_squared_error(test_1['Weekly_Sales'][6:], pred.flatten()))

1.0154105598483505

# Overall Result

| Model | Train RMSE | Test RMSE |
|------|-------------|-----------|
| Logistic Regression | 1.90 | 1.86 | 
| RandomForest Regressor | 0.76 | 1.76 |
| Neural Network | 0.73 | 0.76 |
